# 相関比

- 数量データ　と　カテゴリデータ
    - 指標：相関比
    - 取りうる値：0~1
    - 計算式
        
        $$
        \dfrac{級間変動}{級内変動+級間変動}
        $$
        

（値-値の平均）^2

これを全部足すと，級内変動

データの個数×（値の平均ー全体の平均）^2

これを全部足すと級間変動

1~0.8　非常に強い

0.8~0.5　やや強い

0.5~0.25　やや弱い　

0.25未満　関連していない

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib

In [ ]:
# 1. データの準備
# 例：とあるアパレル店の「店舗（カテゴリー）」と「売上（数値）」の関係
data = {
    '店舗': ['A店']*5 + ['B店']*5 + ['C店']*5,
    '売上': [
        # A店: 平均的
        100, 102, 98, 105, 95,
        # B店: 売上が高い
        150, 155, 145, 160, 140,
        # C店: 売上が低い
        50, 55, 45, 60, 40
    ]
}
df = pd.DataFrame(data)

In [ ]:
# 2. 必要な統計量の計算

# (1) 全体の平均
total_mean = df['売上'].mean()

# (2) 全変動 (S_total): 全データと全体平均との差の2乗和
# 数式: Σ(x - x_bar)^2
s_total = ((df['売上'] - total_mean) ** 2).sum()

# (3) 級間変動 (S_between): 「各グループの平均」と「全体平均」との差の2乗和（重み付き）
# まず、店舗ごとの平均とデータ数(count)を計算します
group_stats = df.groupby('店舗')['売上'].agg(['mean', 'count'])
# 計算: Σ(データ数 * (グループ平均 - 全体平均)^2)
s_between = (group_stats['count'] * ((group_stats['mean'] - total_mean) ** 2)).sum()

# (4) 相関比 (eta) の計算
# 相関比の2乗 (eta squared) = 級間変動 / 全変動
eta_squared = s_between / s_total

# 相関比 (eta) = 平方根をとる
eta = np.sqrt(eta_squared)

# --- 結果の表示 ---
print(f"--- 計算結果 ---")
print(f"全変動 (S_total)  : {s_total:.2f}")
print(f"級間変動 (S_between): {s_between:.2f}")
print(f"相関比 (η)        : {eta:.4f}")

if eta > 0.7:
    print("判定: 強い相関があります（店舗によって売上が明確に違う）。")
elif eta > 0.4:
    print("判定: 中程度の相関があります。")
else:
    print("判定: 相関は弱いです（店舗による違いはあまりない）。")

In [ ]:
# --- 3. 可視化（箱ひげ図） ---
# カテゴリー×数値の関係を見るには「箱ひげ図」がベストです
plt.figure(figsize=(8, 6))
sns.boxplot(x='店舗', y='売上', data=df, palette='Set2')
# データの実点も重ねて表示（分布がわかりやすくなります）
sns.stripplot(x='店舗', y='売上', data=df, color='black', alpha=0.5)

plt.title(f'店舗ごとの売上の違い (相関比 η={eta:.2f})')
plt.grid(axis='y', alpha=0.5)
plt.show()